In [17]:
import re
import os
import random
import numpy as np

from collections import namedtuple

## TREC - Question Answering (multi-class)

In [18]:
# TREC - Question Answering
# http://cogcomp.cs.illinois.edu/Data/QA/QC/

TREC_Question = namedtuple("TREC_Question", "label question")

trec_train = set()
trec_test = set()

for filename in os.listdir("TREC/"):
    with open("TREC/"+filename,'r', encoding='latin_1') as f_input:
        for line in f_input:
            label, question = line.split(' ', 1)
            label = label.split(':')[0]
            question = TREC_Question(label, question)
            if filename=='TREC_10.label':
                trec_test.add(question)
            else:
                trec_train.add(question)

### Samples and Classes/Labels

In [19]:
print("Train Samples: {}".format(len(trec_train)))
print("Test Samples : {}".format(len(trec_test)))
print("Labels       : {}".format({x.label for x in trec_train}))

Train Samples: 5381
Test Samples : 500
Labels       : {'ENTY', 'DESC', 'ABBR', 'LOC', 'HUM', 'NUM'}


## Stanford Sentiment Treebank - movie reviews with fine-grained labels

In [20]:
# Stanford Sentiment Treebank - movie reviews with fine-grained labels
# https://nlp.stanford.edu/sentiment/

ST_sentence = namedtuple("Stanford_Sentiment", "id sentence")
ST_score = namedtuple("Stanford_Sentiment", "id score")

sentences = []
scores = []
train = []
dev = []
test = []

for filename in ['datasetSentences.txt','datasetSplit.txt', 'sentiment_labels.txt']:
    with open("ST/"+filename,'r') as f_input:
        for line in f_input:
            
            # skip headers
            if line.startswith("sentence_index") or line.startswith('phrase id'):
                    continue
            
            # load sentences
            if filename=='datasetSentences.txt':                                
                sent_id, sentence = line.split('\t', 1)
                sample = ST_sentence(sent_id, sentence)
                sentences.append(sample)
            
            # load splits
            if filename=='datasetSplit.txt':
                sent_id, split = line.split(',', 1)
                split = int(split.strip())
                if split == 1:                    
                    train.append(sent_id)
                if split == 2:                                        
                    test.append(sent_id)
                if split == 3:
                    dev.append(sent_id)
            
            # sentences_id        
            if filename=='sentiment_labels.txt':                    
                sent_id, sent_score = line.split('|', 1)
                #sent_score = float(sent_score.strip())
                sample = ST_score(sent_id, float(sent_score.strip()))
                scores.append(sample)

### Samples and Classes/Labels

In [21]:
print("Total Nr. Samples: {}".format(len(sentences)))
print("Total Nr. Scores : {}".format(len(scores)))
print()
print("Train  : {}".format(len(train)))
print("Dev    : {}".format(len(dev)))
print("Test   : {}".format(len(test)))

Total Nr. Samples: 11855
Total Nr. Scores : 239232

Train  : 8544
Dev    : 1101
Test   : 2210


In [22]:
{y.score for y in scores}

{0.0,
 0.013889,
 0.027778,
 0.041667,
 0.055556,
 0.069444,
 0.083333,
 0.097222,
 0.11111,
 0.125,
 0.13542,
 0.13889,
 0.14583,
 0.15278,
 0.16667,
 0.17708,
 0.18056,
 0.19444,
 0.19792,
 0.20833,
 0.21875,
 0.22222,
 0.23611,
 0.23958,
 0.24306,
 0.25,
 0.26042,
 0.26389,
 0.27083,
 0.27778,
 0.28125,
 0.29167,
 0.30208,
 0.30556,
 0.3125,
 0.31944,
 0.33333,
 0.34375,
 0.34722,
 0.35417,
 0.36111,
 0.36458,
 0.375,
 0.38194,
 0.38542,
 0.38889,
 0.39583,
 0.40278,
 0.40625,
 0.41667,
 0.42361,
 0.42708,
 0.43056,
 0.4375,
 0.44444,
 0.44792,
 0.45139,
 0.45833,
 0.46875,
 0.47222,
 0.47917,
 0.48611,
 0.48958,
 0.49306,
 0.5,
 0.50694,
 0.51042,
 0.51389,
 0.52083,
 0.52778,
 0.53125,
 0.54167,
 0.54861,
 0.55208,
 0.55556,
 0.5625,
 0.56944,
 0.57292,
 0.58333,
 0.59028,
 0.59375,
 0.59722,
 0.60417,
 0.61111,
 0.61458,
 0.625,
 0.63194,
 0.63542,
 0.63889,
 0.64583,
 0.65278,
 0.65625,
 0.66667,
 0.67708,
 0.68056,
 0.6875,
 0.69444,
 0.69792,
 0.70833,
 0.71875,
 0.72222,
 0.7

## TODO: transform to classes or also use as regression problem

In [6]:
# (very positive, positive, neutral, negative, very negative), re-labeled by Socher et al. (2013).4

In [7]:
# 0 - 2.0 
# 2.0 - 4.0
# 4.0 - 6.0
# 6.0 - 8.0
# 8.0 - 10.0

## Movie reviews with one sentence per review

In [8]:
# Movie reviews with one sentence per review.
# https://www.cs.cornell.edu/people/pabo/movie-review-data/

# fold 1: files tagged cv000 through cv099, in numerical order
# fold 2: files tagged cv100 through cv199, in numerical order
# fold 10: files tagged cv900 through cv999, in numerical order

folds = [set() for _ in range(10)]
Review = namedtuple("MovieReview", "label sentence")

for directory in ['MR/txt_sentoken/pos/','MR/txt_sentoken/neg/'] :
    for filename in os.listdir(directory) + os.listdir(directory):
        label = directory.split("/")[-2]        
        fold = int(filename.split("_")[0][2])
        with open(directory+filename,'r') as f_input:
            for line in f_input:
                sample = Review(label,line.strip())
                folds[fold].add(sample)

for i, x in enumerate(folds):
    print(i, len(x))

0 6298
1 6138
2 6162
3 6357
4 6655
5 6589
6 6883
7 6551
8 6193
9 6586


In [9]:
{msg.label for fold in folds for msg in fold}

{'neg', 'pos'}

## Customer Review

In [10]:
# CR: http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html

CustomerReview = namedtuple("Customer_Review", "aspects sentence")
data = []

for filename in os.listdir('CR/'):
    if filename=='Readme.txt':
        continue
    print(filename)
    with open('CR/'+filename,'r') as f_input:
        for line in f_input:
            if line.startswith("*"):
                continue
            # select only lines with an opinion over a feature of the product
            m = re.match(r'.*\[(.[0-9])\].*##.*',line)
            if m:
                sentence = line.split('##')[1].strip()
                aspects_string = line.split('##')[0]
                aspects = aspects_string.split(",")
                data.append(CustomerReview(aspects, sentence))
print(len(data))

Apex AD2600 Progressive-scan DVD player.txt
Canon G3.txt
Creative Labs Nomad Jukebox Zen Xtra 40GB.txt
Nikon coolpix 4300.txt
Nokia 6610.txt
1727


In [11]:
for msg in data:
    if len(msg.aspects) > 1:
        print(msg.aspects)

['player[+2]', 'sound[-1]']
['look[+3]', 'panel button layout[+3]', 'feature[+2]']
['forward[+2]', ' rewind[+2]']
['play[+2]', ' dvd-r[+1]']
['remote control[-1]', ' mp3 filename[-1]']
['play[-2]', ' disney movie[-2]']
['player[+1]', ' look[+3]']
['customer service[-3]', ' technical support[-3]']
['dvd[-2]', ' read[-2]', ' play[-2]']
['picture quality[+2]', 'feature[+2]']
['look[+2]', ' feature[+2]']
['play[-2]', ' dvd[-2]']
['jpeg slideshow[+2]', 'mpeg1[+1]']
['picture[-2]', ' player[-3][p]']
['play[-2]', ' disc[-2]']
['play[-2]', ' dvd[-2]']
['picture[-2]', ' player[-2][p]']
['dvd[-2]', ' play[-2]']
['dvd[-2]', ' play[-2]']
['format[+2][u]', 'progressive scan[+2]']
['play[+2]', ' different file[+2]']
['play[+2]', ' mpeg[+2]']
['no disc[-2]', ' screen[-3][u]']
['player[+3]', ' format[+2]']
['progressive scan[+2]', 'remote[+3]']
['play[-2]', ' dvd[-2]', ' no disc[-2]']
['look[+2][u]', 'panel[+2]']
['dvd[-2]', ' picture[-2]', ' play[-2]']
['picture[-2]', ' sound[-2]']
['dvd[-2]', ' read

## Prepare training data for input to neural network

In [12]:
max_len = max([len(x.question.split()) for x in trec_train])
questions_train = [x.question for x in trec_train]
labels_train = [x.label for x in trec_train]

# Convert words to indexes
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions_train)
sequences_train = tokenizer.texts_to_sequences(questions_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# Pad the sequences to the 'max_len'
data_train = pad_sequences(sequences_train, maxlen=max_len)

# Encode the labels, each must be a vector with dim = num. of possible labels
le = LabelEncoder()
le.fit(labels_train)
labels_encoded_train = le.transform(labels_train)
categorical_labels_train = to_categorical(labels_encoded_train, num_classes=None)
print('Shape of train data tensor:', data_train.shape)
print('Shape of train label tensor:', categorical_labels_train.shape)

# Shuffle training data
indices = np.arange(data_train.shape[0])
np.random.shuffle(indices)
x_train = data_train[indices]
y_train = categorical_labels_train[indices]

Found 8461 unique tokens.
Shape of train data tensor: (5381, 37)
Shape of train label tensor: (5381, 6)


In [13]:
# Pre-Process test data
questions_test = [x.question for x in trec_test]
y_test = [x.label for x in trec_test]
sequences_test = tokenizer.texts_to_sequences(questions_test)
x_test = pad_sequences(sequences_test, maxlen=max_len)

print('Shape of test data tensor:', x_test.shape)

Shape of test data tensor: (500, 37)
